In [1]:
import numpy as np
import pandas as pd
import xmltodict
import glob
import re
from os.path import basename, splitext

/home/me/.local/share/virtualenvs/src-LO6I0dqR/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
data_dir = glob.glob('../../Data/Training/*')
df = {}

In [3]:
for genre_file in data_dir:
    with open(genre_file) as f:
        tmp = f.read().split('</doc>')
        name = splitext(basename(genre_file))[0]
        df[name] = tmp
        del tmp

In [4]:
tmp = []
for name, corpus in df.items():
    i = 0
    for item in corpus:
        m = re.search('<doc id="[0-9]+" genre="[a-zA-Z]+" gender="[M|F]">', item)
        if m is not None:
            i += 1
            tmp.append((m.group(0), item))
    assert i==len(corpus)-1
del df
df = pd.DataFrame(tmp)


In [21]:
df.rename(columns={0:'meta', 1:'text'}, inplace=True)
df['gender'] = df.iloc[:,0].str.extract(r'(M|F)')
df['genre'] = df.iloc[:,0].str.extract(r'(twitter|youtube|diary|journalism|children)')
df['docid'] = df.iloc[:,0].str.extract(r'([0-9]+)')
df.sample(5)

,meta,text,gender,genre,docid
5555,"<doc id=""4251"" genre=""twitter"" gender=""M"">","Vedrai, dovra pagare i danni alla veturs https...",M,twitter,4251
1874,"<doc id=""4926"" genre=""youtube"" gender=""F"">",Ma nn ti fai schifo?,F,youtube,4926
145,"<doc id=""146"" genre=""diary"" gender=""F"">",11 luglio 1983 (Io e Robi)\nCon Robert ho molt...,F,diary,146
8992,"<doc id=""4511"" genre=""twitter"" gender=""F"">",@ComeVePare ma purtroppo su snapchat ho la gen...,F,twitter,4511
2229,"<doc id=""4134"" genre=""youtube"" gender=""M"">",Commentare Salvini è pura perdita di tempo. D'...,M,youtube,4134


In [30]:
for idx, row in df.iterrows():
    row.text = row.text.replace(row.meta, '').strip()

In [31]:
df.to_json('../../Data/gxg.json', orient='records', lines=True)

In [33]:
pd.read_json('../../Data/gxg.json', lines=True).sample(5)

,docid,gender,genre,meta,text
10476,3758,M,twitter,"<doc id=""3758"" genre=""twitter"" gender=""M"">",@matteorenzi agli #SGT2016 dice che al #Sud ma...
6889,1581,M,twitter,"<doc id=""1581"" genre=""twitter"" gender=""M"">",Le nostre spiagge italiane .... http:// fb.me/...
10176,4120,M,twitter,"<doc id=""4120"" genre=""twitter"" gender=""M"">",Il #Barça e quel peccato chiamato presunzione ...
6329,590,M,twitter,"<doc id=""590"" genre=""twitter"" gender=""M"">",Se avessimo avuto i cambi #RomaInter #Amala
3785,7296,F,youtube,"<doc id=""7296"" genre=""youtube"" gender=""F"">",HAHAHAHHAHAHA


In [44]:
%%time

import numpy as np
import pandas as pd
import sys
# Add the ptdraft folder path to the sys.path list
sys.path.append('../')
from labov import datasets, classifier, evaluation
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils import shuffle

x, y = shuffle(df.text, df.gender, random_state=42)
print("running the baseline...")
random = cross_val_score(classifier.random, x, y, cv=5, n_jobs=-1)
print("done!")
print("Accuracy: %0.2f (+/- %0.2f)" % (random.mean(), random.std() * 2))

print("fitting model...")
y_pred = cross_val_predict(classifier.ngram, x, y, cv=10, n_jobs=-1)
print("done!")
print(evaluation.run(y_pred, y))

running the baseline...
done!
Accuracy: 0.51 (+/- 0.02)
fitting model...
done!
0.6836363636363636
             precision    recall  f1-score   support

          F       0.67      0.69      0.68      5328
          M       0.70      0.68      0.69      5672

avg / total       0.68      0.68      0.68     11000

[[3674 1654]
 [1826 3846]]
0
CPU times: user 425 ms, sys: 148 ms, total: 574 ms
Wall time: 1min 25s
